In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - LLaVA Deployment

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_llava.ipynb">
      <img alt="Workbench logo" src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" width="32px"><br> Run in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_pytorch_llava.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_llava.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates how to deploy a **Pytorch-Llava** open model on Google Cloud Vertex AI.

### Objectives

- Deploy Pytorch-Llava using containerized backends like [vLLM](https://github.com/vllm-project/vllm) on GPU.
- Use the deployed model to serve chat completion requests for both text and multimodal inputs.

### File a Bug

If you encounter issues with this notebook, report them on [GitHub](https://github.com/GoogleCloudPlatform/vertex-ai-samples/issues/new).

### Costs

This tutorial uses billable components of Google Cloud:

- Vertex AI
- Cloud Storage

Refer to the [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing) pages for more information. Use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to estimate your projected costs.

## Get Started

### Install Vertex AI SDK and other required packages

In [ ]:
%pip install --upgrade --force-reinstall --quiet 'google-cloud-aiplatform>=1.106.0' 'openai' 'google-auth==2.27.0' 'requests==2.32.3'

### Authenticate the Notebook Environment (Colab only)

If you're running this notebook in Google Colab, run the following cell to authenticate.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud Project Information

To get started with Vertex AI, ensure you have an existing Google Cloud project and that the [Vertex AI API is enabled](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

See the guide on [setting up your project and development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment). Also confirm that [billing is enabled](https://cloud.google.com/billing/docs/how-to/modify-project).


In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os

import vertexai

PROJECT_ID = ""  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}

if not PROJECT_ID:
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")

REGION = ""  # @param {type: "string", placeholder: "[your-region]", isTemplate: true}

if not REGION:
    REGION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=REGION)

print(f"Project: {PROJECT_ID}\nLocation: {REGION}")

### Import libraries

In [ ]:
from vertexai import model_garden

## Deploy model

### Choose model variant

You can proceed with the default model variant or select a different one.

In [ ]:
model_version = "llava-1.5-7b-hf"  # @param ["llama3-llava-next-8b-hf", "llava-1.5-13b-hf", "llava-1.5-7b-hf", "llava-v1.5-7b", "llava-v1.6-34b-hf", "llava-v1.6-mistral-7b-hf", "llava-v1.6-vicuna-13b-hf", "llava-v1.6-vicuna-7b-hf"] {isTemplate:true}
MODEL_NAME = f"liuhaotian/pytorch-llava@{model_version}"

To see all deployable model variants available in Model Garden, use:

In [ ]:
all_model_versions = model_garden.list_deployable_models(
    model_filter="pytorch-llava", list_hf_models=False
)

Once you've selected a model variant, initialize it:

In [ ]:
model = model_garden.OpenModel(MODEL_NAME)

### Check the Deployment Configuration

Use the `list_deploy_options()` method to view the verified deployment configurations for your selected model. This helps ensure you have sufficient resources (e.g., GPU quota) available to deploy it.

In [ ]:
deploy_options = model.list_deploy_options(concise=True)
print(deploy_options)

### Deploy the Model

Now that you’ve reviewed the deployment options, use the `deploy()` method to serve the selected open model to a Vertex AI endpoint. Deployment time may vary depending on the model size and infrastructure requirements.

> **Note**: If the model requires accepting a license agreement (EULA), set the `accept_eula=True` flag in the deploy call. Set `use_dedicated_endpoint` to False if you don't want to use [dedicated endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment#create-dedicated-endpoint).

In [ ]:
use_dedicated_endpoint = True

In [ ]:
endpoints = {}

In [ ]:
endpoints["sdk_default"] = model.deploy(
    accept_eula=True,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

Alternatively, you can select one of the verified deployment configurations listed above.

In [ ]:
endpoints["sdk_custom"] = model.deploy(
    accept_eula=True,
    use_dedicated_endpoint=use_dedicated_endpoint,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20250601_0916_RC01",
    machine_type="g2-standard-12",
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,
)

In [ ]:
if "sdk_default" in endpoints:
    endpoint = endpoints["sdk_default"]
    LABEL = "sdk_default"
elif "sdk_custom" in endpoints:
    endpoint = endpoints["sdk_custom"]
    LABEL = "sdk_custom"
else:
    raise ValueError("No endpoint found. Create an endpoint.")

To further customize your deployment, you can configure:

- **Compute Resources**: Machine type, replica count (min/max), accelerator type and quantity.
- **Infrastructure**: Use Spot VMs, reservation affinity, or dedicated endpoints.
- **Serving Container**: Customize container image, ports, health checks, and environment variables.

See the [Model Garden SDK README](https://github.com/googleapis/python-aiplatform/blob/main/vertexai/model_garden/README.md) for advanced configuration options.

In [ ]:
# @title Predict

# @markdown This section uses the deployed LLaVA model to answer questions about a given image.
# fmt: off
image_url = "https://llava-vl.github.io/static/images/view.jpg"  # @param {type:"string"}
# fmt: on

# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint.name` allows us to get the
#   endpoint name of the endpoint `endpoint` created in the cell
#   above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint = aiplatform.Endpoint(aip_endpoint_name)

raw_prompt = "What's in the image?"  # @param {type: "string"}
prompt = f"USER: <image>{raw_prompt}ASSISTANT:"
# @markdown If you encounter the issue like `ServiceUnavailable: 503 Took too long to respond when processing`, you can reduce the maximum number of output tokens, such as set `max_tokens` as 20.
max_tokens = 256  # @param {type:"integer"}
temperature = 0.8  # @param {type:"number"}
top_p = 0.95  # @param {type:"number"}

# Overrides parameters for inferences.
instances = [
    {
        "prompt": prompt,
        "multi_modal_data": {"image": image_url},
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
    },
]
response = endpoint.predict(instances=instances)

for raw_prediction in response.predictions:
    prediction = raw_prediction.split("Output:")
    print(prediction[1])

# @markdown Click "Show code" to see more details.

## Clean up resources

In [ ]:
# @title Delete the models and endpoints
# @markdown Delete the endpoint.

for endpoint in endpoints.values():
    endpoint.delete(force=True)